<a href="https://colab.research.google.com/github/KinaniJ/Whatsapp_Data/blob/main/whatsapp_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
#importing libraries
import pandas as pd
import numpy as np
from google.colab import files
upload = files.upload()


Saving WhatsApp Chat with Phoenix Analytics 2.0.txt to WhatsApp Chat with Phoenix Analytics 2.0.txt


In [ ]:
with open(f"WhatsApp Chat with Phoenix Analytics 2.0.txt", encoding="utf-8") as f:
    # Read the contents of the file into a variable
    chat = f.read() # Changed variable name to avoid confusion with the DataFrame

# Split the chat into lines
lines = chat.splitlines()

# Create a list to store the data
data = []
for line in lines:
    data.append(line)

# Create the DataFrame from the list of lines
df = pd.DataFrame(data, columns=['chat_lines']) # Added column name
df.head()

In [211]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2674 entries, 0 to 2673
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   chat_lines  2674 non-null   object
dtypes: object(1)
memory usage: 21.0+ KB


In [212]:
def clean_chat(df):
  """

  Args:
    df: a dataframe with one columns which is line chats

  splits the one column into date, time, activities, and the actual chats

  Returns: (df) a clean dataframe with 5 columns

  """
  df['chat_lines'] = df[~df['chat_lines'].str.contains('Your security code with|change to', case = False, na = False)] #remove lines with the string as they are not part of the chats
  df['date'] = df['chat_lines'].str.split(',', expand = True)[0] #extract date
  df['date'] = pd.to_datetime(df['date'], dayfirst = True, errors = 'coerce') #change date into datetime datatype
  df['time'] = df['chat_lines'].str.extract(r'\d{1,2}/\d{1,2}/\d{2},\s([\d:]+\s?[APM]*)\s-') #extract time
  df['time'] = pd.to_datetime(df['time'], format='%I:%M %p', errors = 'coerce').dt.time #change time into datetime datatype
  df['contact'] = df['chat_lines'].str.extract(r'(-\s(\+[\d\s]+)|-\s([A-Za-z]+):)')[0] #extract contatcs
  conditions = [
        df['chat_lines'].str.contains('Media omitted', case = False, na = False),
        df['chat_lines'].str.contains('poll', case = False, na = False),
        df['chat_lines'].str.contains('removed', case = False, na = False),
        df['chat_lines'].str.contains('joined', case = False, na = False)
        ] #group chats into normal chats, polls, removed and joined
  choices = ['Media', 'poll', 'removed', 'joined']
  df['activity'] = np.select(conditions, choices, default = 'chat')
  df['chat'] = df['chat_lines'].str.split(':', expand = True)[2] #extract chats
  df.drop(columns = ['chat_lines'], inplace = True) #drop the orginal text column
  df.dropna(subset = ['time', 'contact', 'activity', 'chat'],  inplace = True) #remove rows with null values for all columns apart from date

  return df

In [ ]:
#call the function inspect the data
clean_chat(df)
df.head()

In [214]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1603 entries, 3 to 2673
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      1603 non-null   datetime64[ns]
 1   time      1603 non-null   object        
 2   contact   1603 non-null   object        
 3   activity  1603 non-null   object        
 4   chat      1603 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 75.1+ KB


In [217]:
#save the data in a csv
df.to_csv(r"phoenix_whatsapp_clean_chats.csv", encoding='utf-8') #changed the encoding to utf-8 which supports a broader range of characters

